# Simple Linear Regression with Gradient Descent in PyTorch

This notebook will guide you through the implementation of a simple linear regression model using PyTorch.  
We will use gradient descent for optimization and employ a custom visualization class to help you understand the model's learning process both in the data space and in the parameter (weight/bias) space.

---

## Table of Contents

1. [Introduction](#introduction)
2. [Importing Libraries](#importing-libraries)
3. [Error Surface Visualization Class](#error-surface-visualization-class)
4. [Creating a Random Dataset](#creating-a-random-dataset)
5. [Defining the Model and Loss Function](#defining-the-model-and-loss-function)
6. [Training the Linear Regression Model](#training-the-linear-regression-model)
7. [Experiment: Higher Learning Rate](#experiment-higher-learning-rate)
8. [Summary](#summary)

---

## Introduction

Linear regression is one of the simplest and most fundamental algorithms in machine learning.  
It models the relationship between a dependent variable $y$ and an independent variable $x$ by fitting a straight line ($y = wx + b$).  
The goal is to find the optimal values for weight ($w$) and bias ($b$) that minimize the prediction error on the training data.

In this notebook, we will:
- Generate synthetic data that follows a linear relationship with some random noise.
- Implement a simple linear regression model from scratch using PyTorch tensors.
- Use gradient descent to iteratively update the model's parameters and minimize the mean squared error.
- Visualize both the data and the learning process, including the error surface in $w$/$b$ parameter space.

---

## Importing Libraries

First, we import all the libraries that are required for numerical operations, visualization, and PyTorch.

```python
# Utilities for Jupyter/IPython (not strictly necessary for the code to run)
from IPython import get_ipython
from IPython.display import display

# Numpy is used for numerical operations and efficient array handling
import numpy as np

# Matplotlib is the go-to Python library for plotting and visualization
import matplotlib.pyplot as plt

# mpl_toolkits.mplot3d is required for 3D plotting (used for visualizing error surfaces)
from mpl_toolkits import mplot3d
```

---

## Error Surface Visualization Class

To better understand how gradient descent works, it's extremely helpful to visualize the "error surface"—that is, how the loss (mean squared error) changes as we vary the model parameters ($w$ and $b$).

The `plot_error_surfaces` class below constructs a 3D surface (and 2D contour) showing the loss for different values of $w$ and $b$ on your data. It also keeps track of the historical parameter updates during training and can plot the path that gradient descent takes.

**Key features:**
- Computes and stores the loss (mean squared error) for a grid of $w$ and $b$ values.
- Plots 3D surfaces and 2D contours of the loss.
- Tracks the path of parameter updates during training.

```python
class plot_error_surfaces(object):    
    def __init__(self, w_range, b_range, X, Y, n_samples = 30, go = True):
        """
        Initializes the visualization class by creating a meshgrid of w and b values,
        computing the mean squared error at each point, and plotting the initial surfaces.
        
        Args:
            w_range (float): The range for the weight parameter w, from -w_range to +w_range.
            b_range (float): The range for the bias parameter b, from -b_range to +b_range.
            X (Tensor): The input features (PyTorch tensor).
            Y (Tensor): The target values (PyTorch tensor).
            n_samples (int): Number of grid samples along each axis for the surface.
            go (bool): If True, plot the surfaces immediately.
        """
        W = np.linspace(-w_range, w_range, n_samples)
        B = np.linspace(-b_range, b_range, n_samples)
        w, b = np.meshgrid(W, B)    
        Z = np.zeros((n_samples, n_samples))
        self.y = Y.numpy()
        self.x = X.numpy()
        # Compute the MSE at each (w, b) pair in the grid
        for i in range(n_samples):
            for j in range(n_samples):
                Z[i, j] = np.mean((self.y - (w[i, j] * self.x + b[i, j])) ** 2)
        self.Z = Z
        self.w = w
        self.b = b
        self.W = []    # To store the sequence of w values during training
        self.B = []    # To store the sequence of b values during training
        self.LOSS = [] # To store the sequence of loss values during training
        self.n = 0     # Counter for number of training steps
        if go:
            # 3D surface plot of the error surface
            plt.figure(figsize=(7.5, 5))
            ax = plt.axes(projection='3d')
            ax.plot_surface(self.w, self.b, self.Z, rstride=1, cstride=1, cmap='viridis', edgecolor='none')
            plt.title('Cost/Total Loss Surface')
            plt.xlabel('w')
            plt.ylabel('b')
            plt.show()
            # 2D contour plot of the error surface
            plt.figure()
            plt.title('Cost/Total Loss Surface Contour')
            plt.xlabel('w')
            plt.ylabel('b')
            plt.contour(self.w, self.b, self.Z)
            plt.show()
    
    def set_para_loss(self, W, B, loss):
        """
        Record the current parameter values and loss.
        Call this after each update step in training for visualization.
        
        Args:
            W (float): Current value of weight parameter.
            B (float): Current value of bias parameter.
            loss (float): Current loss value.
        """
        self.n += 1
        self.W.append(W)
        self.B.append(B)
        self.LOSS.append(loss)
    
    def final_plot(self):
        """
        After training, visualize the path of parameter updates on the error surface.
        Shows both a 3D wireframe and a 2D contour plot.
        """
        # 3D plot: parameter path on the error surface
        ax = plt.axes(projection='3d')
        ax.plot_wireframe(self.w, self.b, self.Z)
        ax.scatter(self.W, self.B, self.LOSS, c='r', marker='x', s=200, alpha=1)
        plt.title("Training Path on Error Surface (3D)")
        plt.xlabel('w')
        plt.ylabel('b')
        plt.show()
        # 2D contour plot: parameter path
        plt.figure()
        plt.contour(self.w, self.b, self.Z)
        plt.scatter(self.W, self.B, c='r', marker='x')
        plt.xlabel('w')
        plt.ylabel('b')
        plt.title("Training Path on Error Surface (Contour)")
        plt.show()
    
    def plot_ps(self):
        """
        At each training step, plot:
        - The data space (showing the model's current fit versus the data)
        - The parameter space (showing the trajectory of parameters on the error surface contour)
        """
        plt.figure(figsize=(12, 5))
        # Data space (left): how well do current w/b fit the data?
        plt.subplot(1, 2, 1)
        plt.ylim((-10, 15))
        plt.plot(self.x, self.y, 'ro', label="Training Points")
        plt.plot(self.x, self.W[-1] * self.x + self.B[-1], label="Estimated Line")
        plt.xlabel('x')
        plt.ylabel('y')
        plt.title(f'Data Space Iteration: {self.n}')
        plt.legend()
        # Parameter space (right): path on error surface
        plt.subplot(1, 2, 2)
        plt.contour(self.w, self.b, self.Z)
        plt.scatter(self.W, self.B, c='r', marker='x')
        plt.title(f'Total Loss Surface Contour Iteration {self.n}')
        plt.xlabel('w')
        plt.ylabel('b')
        plt.tight_layout()
        plt.show()
```

---

## Creating a Random Dataset

To demonstrate linear regression, we'll generate a simple synthetic dataset.  
We'll create input values $x$ from $-3$ to $3$, and generate outputs $y$ according to the true linear relationship $y = 1 \cdot x - 1$, plus some random Gaussian noise to simulate real-world data.

```python
import torch  # PyTorch is used for tensors and automatic differentiation

# Create input data: 1D tensor from -3 to 3 (step 0.1), reshaped as a column vector
X = torch.arange(-3, 3, 0.1).view(-1, 1)

# True function: y = x - 1
f = 1 * X - 1

# Add noise: Normal(0, 0.1) noise added to each y value
Y = f + 0.1 * torch.randn(X.size())

# Visualize the noisy data and the true line
plt.figure(figsize=(8, 5))
plt.plot(X.numpy(), Y.numpy(), 'ro', label='Noisy Data')
plt.plot(X.numpy(), f.numpy(), label='True Line')
plt.title('Generated Data: Noisy Points and True Linear Relationship')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()
```

---

## Defining the Model and Loss Function

For linear regression, our prediction is:
$$
\hat{y} = w \cdot x + b
$$
where $w$ and $b$ are the parameters we want to learn.

The **forward function** computes predictions for a given $x$.  
The **criterion** function computes the mean squared error (MSE) loss between predictions and true values.

```python
def forward(x):
    """
    Compute the predicted output y_hat for input x using current w and b.
    """
    return w * x + b

def criterion(yhat, y):
    """
    Compute mean squared error between predictions (yhat) and true labels (y).
    """
    return torch.mean((yhat - y) ** 2)
```

### Visualizing the Error Surface

Before training, let's initialize the visualization class to view the error surface
for our synthetic dataset.

```python
# This will plot the loss surface for a grid of (w, b) values
get_surface = plot_error_surfaces(15, 15, X, Y, 30)
```

---

## Training the Linear Regression Model

Now we set up model parameters, the learning rate, and implement the training loop using gradient descent.

- **Parameters**: $w$ (weight) and $b$ (bias), both initialized far from the optimum for demonstration.
- **Learning Rate**: Controls the step size of each parameter update.
- **Training Loop**: For a fixed number of epochs, we:
  - Compute predictions and loss
  - Record parameter values/loss for visualization
  - Compute gradients via backpropagation
  - Update parameters using gradient descent
  - Zero gradients before next step

```python
# Initialize model parameters (w and b), with gradients enabled
w = torch.tensor(-15.0, requires_grad=True)
b = torch.tensor(-10.0, requires_grad=True)

# Learning rate (step size for gradient descent)
lr = 0.1

# Track the loss at each iteration for plotting
LOSS = []

def train_model(iter):
    """
    Train the model using gradient descent.
    
    Args:
        iter (int): Number of epochs/iterations to train for.
    """
    for epoch in range(iter):
        Yhat = forward(X)                 # Predictions
        loss = criterion(Yhat, Y)         # Loss (MSE)
        get_surface.set_para_loss(w.data.tolist(), b.data.tolist(), loss.tolist()) # Record for visualization
        if epoch % 3 == 0:
            get_surface.plot_ps()         # Show fit and parameter path every 3 epochs
        LOSS.append(loss)
        loss.backward()                   # Compute gradients
        # Gradient descent update (in-place, so PyTorch doesn't track this step)
        w.data = w.data - lr * w.grad.data
        b.data = b.data - lr * b.grad.data
        # Zero gradients for next step
        w.grad.data.zero_()
        b.grad.data.zero_()

# Train for 15 epochs
train_model(15)

# Visualize final parameter path on error surface
get_surface.final_plot()

# Plot the loss over epochs
LOSS = [loss.detach().numpy() for loss in LOSS]
plt.plot(LOSS)
plt.tight_layout()
plt.xlabel("Epoch/Iterations")
plt.ylabel("Cost (MSE Loss)")
plt.title("Loss Curve During Training")
plt.show()
```

**What do you see?**  
- The training curve shows how the loss decreases as parameters are updated.
- The error surface plots show how the parameter values ($w, b$) move toward the minimum of the loss surface.

---

## Experiment: Higher Learning Rate

Let's see the effect of using a larger learning rate.

- **Hypothesis:** A higher learning rate may converge faster, but if too high, might overshoot the minimum or cause instability.

```python
# Re-initialize parameters to same (bad) starting values
w = torch.tensor(-15.0, requires_grad=True)
b = torch.tensor(-10.0, requires_grad=True)
lr = 0.2  # Double the learning rate
LOSS2 = []

def my_train_model(iter):
    """
    Train the model using a different learning rate, storing loss in LOSS2.
    """
    for epoch in range(iter):
        Yhat = forward(X)
        loss = criterion(Yhat, Y)
        get_surface.set_para_loss(w.data.tolist(), b.data.tolist(), loss.tolist())
        if epoch % 3 == 0:
            get_surface.plot_ps()
        LOSS2.append(loss)
        loss.backward()
        w.data = w.data - lr * w.grad.data
        b.data = b.data - lr * b.grad.data
        w.grad.data.zero_()
        b.grad.data.zero_()

# Train for 15 epochs with higher lr
my_train_model(15)

# Final visualization
get_surface.final_plot()
LOSS2 = [loss.detach().numpy() for loss in LOSS2]
plt.plot(LOSS2)
plt.tight_layout()
plt.xlabel("Epoch/Iterations")
plt.ylabel("Cost (MSE Loss)")
plt.title("Loss Curve with Higher Learning Rate")
plt.show()
```

**Questions to consider:**
- Does the loss decrease more quickly?
- Is the trajectory on the error surface smoother or more jagged?
- Does increasing the learning rate even more become unstable?

---

## Summary

- We built a basic linear regression model in PyTorch from scratch.
- Used gradient descent to optimize parameters.
- Visualized both the data space (fit to data) and parameter space (error/loss surface and gradient descent path).
- Saw how adjusting the learning rate affects model convergence.

**Further exploration:**
- Try different learning rates, initializations, or more iterations.
- Try adding outliers or more noise to the data.
- Modify the true function (change slope/intercept).
- Extend to multiple features (multivariate regression).

---